In [1]:
{-# LANGUAGE FlexibleContexts #-}

In [2]:
:load src/stats.hs

In [3]:
import Text.CSV
import Stats
import System.Random
import Data.Maybe
import Data.List
import Data.Ord
import Graphics.EasyPlot

In [4]:
goog <- parseCSVFromFile "./data/GOOG.csv"
ford <- parseCSVFromFile "./data/FORD.csv"
amzn <- parseCSVFromFile "./data/AMZN.csv"

In [5]:
length $ bbEither goog

3944

In [6]:
movingSum :: [Double] -> [Double] -> [Double]
movingSum xs [] = [sum xs]
movingSum (x:xs) (y:ys) = (sum (x:xs)):movingSum (xs++[y]) ys

In [7]:
movingSum [1,2,3] [4,5,6,7,8,9,10]

[6.0,9.0,12.0,15.0,18.0,21.0,24.0,27.0]

In [8]:
movingAverage :: [Double] -> Int -> [Double]
movingAverage xs windowSize
    | windowSize < 1 = error "Non positive window size"
    | windowSize > length xs = []
    | otherwise = map (/ fromIntegral windowSize) movingSums
    where
        movingSums = movingSum (take windowSize xs) (drop windowSize xs)

In [9]:
movingAverage [1..10] 3

[2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0]

In [30]:
openGoog :: [Double]
openGoog = readIndex goog 2
openAmzn :: [Double]
openAmzn = readIndex amzn 2
openFord :: [Double]
openFord = readIndex ford 2

In [31]:
Just m = medie openGoog

In [32]:
show m

"510.3194009518249"

In [33]:
take 10 openGoog

[51.835709,54.336334,56.528118,55.591629,53.798351,53.773445,54.107193,52.548038,51.661362,51.292744]

In [34]:
length $ movingAverage openGoog 200

3745

In [35]:
rescale :: [Double] -> [Double]
rescale xs = map (\x -> (x-lead)/(diffxs)) xs
    where
        lead = last xs
        maxxs = maximum xs
        minxs = minimum xs
        diffxs = maxxs - minxs

In [36]:
rescale $ take 5 openGoog

[-0.4182589369340985,0.11464963944958854,0.5817410630659015,0.38216574897882977,0.0]

In [37]:
gen <- newStdGen

In [38]:
values = take 100000 (randomRs (1,100) gen) :: [Double]

In [39]:
:type randomRs

randomRs :: forall g a. (RandomGen g, Random a) => (a, a) -> g -> [a]

In [40]:
chunk [] = []
chunk list = take 100 list : chunk (drop 100 list)

In [41]:
chuncked = chunk values

In [42]:
length chuncked

1000

In [43]:
medii = map medie chuncked

In [44]:
take 10 medii

[Just 50.75506656606628,Just 53.99340115787386,Just 54.82659323042446,Just 49.43293357145652,Just 54.37870197116465,Just 49.7066221550139,Just 49.86507119493165,Just 50.15000658396081,Just 54.515620300852724,Just 52.41760544401462]

In [45]:
pairs = runLengthEncoding $ sort (map (\(Just m) -> round m) medii)

In [46]:
take 5 pairs

[(43,5),(44,11),(45,22),(46,47),(47,57)]

In [49]:
plot X11 $ zip [0,1..] openGoog

True

In [70]:
plot X11 $ Data2D [Style Lines] [] $ zip [0,1..] $ movingAverage openAmzn 100

True

In [73]:
openFord2D = Data2D [Style Lines, Title "Ford Openenings "] [] (zip [0,1..] openFord)
openFordMA2D = Data2D [Style Lines, Title "Ford Avg. Open"] [] (zip [0,1..] (movingAverage openFord 100))
plot X11 [openFord2D, openFordMA2D]

True

In [78]:
openGoog2D = Data2D [Style Lines, Title "GOOG Adj. Open", Color Black] [] (zip [0,1..] $ take 2000 openGoog)
openGoogMA2D = Data2D [Style Lines, Title "GOOG Avg Open", Color Red] [] (zip [100, 101..] $ take 2000 (movingAverage openGoog 100))
plot X11 [openGoog2D, openGoogMA2D]

True

In [82]:
oGoog252 = take 252 openGoog
oFord252 = take 252 openFord
oAmzn252 = take 252 openAmzn

oG2D = Data2D [Style Lines, Title "GOOG", Color Black] [] $ zip [0,1..] $ rescale oGoog252
oF2D = Data2D [Style Lines, Title "FORD", Color Red] [] $ zip [0,1..] $ rescale oFord252
oA2D = Data2D [Style Lines, Title "AMZN", Color Orange] [] $ zip [0,1..] $ rescale oAmzn252
plot X11 [oG2D, oF2D, oA2D]

True

In [98]:
volG :: [Double]
volG = readIndex goog 6
volF :: [Double]
volF = readIndex ford 6
volA :: [Double]
volA = readIndex amzn 6

In [99]:
plot X11 $ zip openGoog volG

True

In [102]:
plot X11 $ zip openGoog $ map log volG

True

In [101]:
plot X11 $ zip openAmzn volA

True

In [104]:
plot X11 [
    Data2D [Title "Goog", Color Red] [] $ zip openGoog $ map log volG,
    Data2D [Title "Ford", Color Blue] [] $ zip openFord $ map log volF,
    Data2D [Title "AMZN", Color Orange] [] $ zip openAmzn $ map log volA
    ]

True